In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import MySQLdb
import mysql.connector

stationinfo_url = "https://gbfs.divvybikes.com/gbfs/en/station_information.json"
stationstatus_url = "https://gbfs.divvybikes.com/gbfs/en/station_status.json"

# Get country information in JSON format
divvy_stationinfo_response = requests.get(stationinfo_url).json()
divvy_stationstatus_response = requests.get(stationstatus_url).json()

# First element is general information, second is countries themselves
divvy_stationinfo_response = divvy_stationinfo_response["data"]["stations"]
divvy_stationstatus_response = divvy_stationstatus_response["data"]["stations"]

divvy_stationinfo_response

[{'station_id': '2',
  'external_id': 'a3a36d9e-a135-11e9-9cda-0a87ae2ba916',
  'name': 'Buckingham Fountain',
  'short_name': '15541',
  'lat': 41.87651122881695,
  'lon': -87.62054800987242,
  'rental_methods': ['CREDITCARD', 'TRANSITCARD', 'KEY'],
  'capacity': 39,
  'electric_bike_surcharge_waiver': False,
  'eightd_has_key_dispenser': False,
  'has_kiosk': True},
 {'station_id': '3',
  'external_id': 'a3a37378-a135-11e9-9cda-0a87ae2ba916',
  'name': 'Shedd Aquarium',
  'short_name': '15544',
  'lat': 41.86722595682,
  'lon': -87.6153553902,
  'rental_methods': ['CREDITCARD', 'TRANSITCARD', 'KEY'],
  'capacity': 55,
  'electric_bike_surcharge_waiver': False,
  'eightd_has_key_dispenser': False,
  'has_kiosk': True},
 {'station_id': '4',
  'external_id': 'a3a378ca-a135-11e9-9cda-0a87ae2ba916',
  'name': 'Burnham Harbor',
  'short_name': '15545',
  'lat': 41.856268,
  'lon': -87.613348,
  'rental_methods': ['CREDITCARD', 'TRANSITCARD', 'KEY'],
  'capacity': 23,
  'electric_bike_surch

In [2]:
divvy_stationstatus_response

[{'station_id': '2',
  'num_bikes_available': 14,
  'num_ebikes_available': 0,
  'num_bikes_disabled': 0,
  'num_docks_available': 25,
  'num_docks_disabled': 0,
  'is_installed': 1,
  'is_renting': 1,
  'is_returning': 1,
  'last_reported': 1576181264,
  'eightd_has_available_keys': False},
 {'station_id': '3',
  'num_bikes_available': 17,
  'num_ebikes_available': 0,
  'num_bikes_disabled': 0,
  'num_docks_available': 38,
  'num_docks_disabled': 0,
  'is_installed': 1,
  'is_renting': 1,
  'is_returning': 1,
  'last_reported': 1576202210,
  'eightd_has_available_keys': False},
 {'station_id': '4',
  'num_bikes_available': 0,
  'num_ebikes_available': 0,
  'num_bikes_disabled': 0,
  'num_docks_available': 23,
  'num_docks_disabled': 0,
  'is_installed': 1,
  'is_renting': 1,
  'is_returning': 1,
  'last_reported': 1576185592,
  'eightd_has_available_keys': False},
 {'station_id': '5',
  'num_bikes_available': 5,
  'num_ebikes_available': 0,
  'num_bikes_disabled': 0,
  'num_docks_avai

In [39]:
divvy_stationinfo_df = pd.DataFrame.from_dict(divvy_stationinfo_response, orient='columns')
divvy_stationinfo_df.reset_index(level=0, inplace=True)
clean_divvy_stationinfo_df = divvy_stationinfo_df.drop(["rental_methods"], axis = 1)
divvy_stationinfo_df.columns

clean_divvy_stationinfo_df

,index,capacity,eightd_has_key_dispenser,electric_bike_surcharge_waiver,external_id,has_kiosk,lat,lon,name,region_id,short_name,station_id
0,0,39,False,False,a3a36d9e-a135-11e9-9cda-0a87ae2ba916,True,41.876511,-87.620548,Buckingham Fountain,NaN,15541,2
1,1,55,False,False,a3a37378-a135-11e9-9cda-0a87ae2ba916,True,41.867226,-87.615355,Shedd Aquarium,NaN,15544,3
2,2,23,False,False,a3a378ca-a135-11e9-9cda-0a87ae2ba916,True,41.856268,-87.613348,Burnham Harbor,NaN,15545,4
3,3,23,False,False,a3a37e26-a135-11e9-9cda-0a87ae2ba916,True,41.874053,-87.627716,State St & Harrison St,NaN,SL-007,5
4,4,39,False,False,a3a38363-a135-11e9-9cda-0a87ae2ba916,True,41.886976,-87.612813,Dusable Harbor,NaN,KA1503000064,6
5,5,19,False,False,a3a3888b-a135-11e9-9cda-0a87ae2ba916,True,41.886349,-87.617517,Field Blvd & South Water St,NaN,15534,7
6,6,15,False,False,a3a39301-a135-11e9-9cda-0a87ae2ba916,True,41.828792,-87.680604,Leavitt St & Archer Ave,NaN,KA1503000068,9
7,7,11,False,False,a3a3983e-a135-11e9-9cda-0a87ae2ba916,True,41.766638,-87.576450,Jeffery Blvd & 71st St,NaN,KA1503000018,11
8,8,15,False,False,a3a39d55-a135-11e9-9cda-0a87ae2ba916,True,41.766409,-87.565688,South Shore Dr & 71st St,NaN,KA1503000002,12
9,9,35,False,False,a3a3a282-a135-11e9-9cda-0a87ae2ba916,True,41.932418,-87.652705,Wilton Ave & Diversey Pkwy,NaN,TA1306000014,13


In [40]:
divvy_stationstatus_df = pd.DataFrame.from_dict(divvy_stationstatus_response, orient='columns')
divvy_stationstatus_df.reset_index(level=0, inplace=True)
divvy_stationstatus_df.head()

,index,eightd_has_available_keys,is_installed,is_renting,is_returning,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,num_ebikes_available,station_id
0,0,False,1,1,1,1576181264,14,0,25,0,0,2
1,1,False,1,1,1,1576202210,17,0,38,0,0,3
2,2,False,1,1,1,1576185592,0,0,23,0,0,4
3,3,False,1,1,1,1576202803,5,0,18,0,0,5
4,4,False,1,1,1,1576190321,0,0,39,0,0,6


In [41]:
redfin_df = pd.read_csv("Redfin_Aug2019.csv")
redfin_df.head()

,City,Neighborhood,Lat,Lng,Month,Sale Price,Homes Sold,New Listings,Days on Market
0,"Chicago, IL",Albany Park,41.968327,-87.728028,Aug-19,295.0,113,133,54
1,"Chicago, IL",Andersonville,41.981658,-87.668476,Aug-19,419.0,94,79,50
2,"Chicago, IL",Archer Heights,41.807900,-87.723585,Aug-19,226.0,20,31,26
3,"Chicago, IL",Archer Highlands,41.878114,-87.629798,Aug-19,210.0,3,5,21
4,"Chicago, IL",Armour Square,41.840755,-87.634019,Aug-19,265.0,7,20,58


In [42]:
database_path = "Project2.db" #whatever you type in here will be the name of the database
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [43]:
inspector = inspect(engine)
inspector.get_table_names() #nice we just created our own db and its empty as it should be!

[]

In [44]:
divvy_stationstatus_df.to_sql(name='divvy_stationstatus', con=engine, if_exists='replace', index=False)
clean_divvy_stationinfo_df.to_sql(name='divvy_stationinfo', con=engine, if_exists='replace', index=False)
redfin_df.to_sql(name='redfin_aug2019', con=engine, if_exists='replace', index=False)

In [45]:
inspector = inspect(engine)
inspector.get_table_names()

['divvy_stationinfo', 'divvy_stationstatus', 'redfin_aug2019']